This script merges ORF candidates from all experiments into one table in the CSV format.
The resulting table will be stored as {local_folder}/merged_orfs_found_by_any_caller.csv
This is the first step of the ORF score prediction and only need to be executed once when you update the experiment list.

In [1]:
import pandas as pd
import os

In [2]:
def download_feature_set(experiment_name, local_path):
    orf_path = f"s3://velia-piperuns-dev/{experiment_name}/output/{experiment_name}_found_by_any_caller.csv"
    local_path = f"{local_folder}/{experiment_name}_found_by_any_caller.csv"
    cmd = f"aws s3 cp {orf_path} {local_path}"
    print(cmd)
    os.system(cmd)
    return local_path

def load_orfs(experiment_name, local_folder, merged_orfs_info, orf_callers):
    data_path = download_feature_set(experiment_name, local_folder)
    if not os.path.exists(data_path):
        return 0
    
    orfs_info = {}
    for index, row in pd.read_csv(data_path, sep='\t').iterrows():
        orf_key = (row["chrom_id"], row["orf_start"], row["orf_end"], row["strand"], row["exon_blocks"])
        if orf_key not in merged_orfs_info:
            merged_orfs_info[orf_key] = row
            merged_orfs_info[orf_key] = pd.concat([merged_orfs_info[orf_key],
                                                   pd.Series({"#called": 1,
                                                              "cell_types": experiment_name})])
        else:
            merged_orfs_info[orf_key]["#called"] += 1
            merged_orfs_info[orf_key]["cell_types"] += ("," + experiment_name)
            for orf_caller in orf_callers:
                if (type(row[f"orf_score_{orf_caller}"]) == type("")
                    and (not row[f"orf_score_{orf_caller}"].strip())):
                    continue
                else:
                    if ((type(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) == type("") and
                         (not merged_orfs_info[orf_key][f"orf_score_{orf_caller}"].strip()))
                        or (merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] is None)
                        or (float(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) <= float(row[f"orf_score_{orf_caller}"]))):
                        merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] = row[f"orf_score_{orf_caller}"]
                        merged_orfs_info[orf_key][f"ORF_type_{orf_caller}"] = row[f"ORF_type_{orf_caller}"]
                        merged_orfs_info[orf_key][f"gene_id_{orf_caller}"] = row[f"gene_id_{orf_caller}"]
                        merged_orfs_info[orf_key][f"transcript_id_{orf_caller}"] = row[f"transcript_id_{orf_caller}"]    
    os.remove(data_path)
    return 1

def merge_orf_info_across_experiments(orfs_info_list, orf_callers):
    merged_orfs_info = {}
    for orfs_info in orfs_info_list:
        for orf_key, orf in orfs_info.items():
            if orf_key not in merged_orfs_info:
                merged_orfs_info[orf_key] = orf
            else:
                for orf_caller in orf_callers:
                    if (type(orf[f"orf_score_{orf_caller}"]) == type("")
                        and (not orf[f"orf_score_{orf_caller}"].strip())):
                        continue
                    else:
                        if ((type(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) == type("") and
                            not merged_orfs_info[orf_key][f"orf_score_{orf_caller}"].strip())
                            or (merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] is None)
                            or (float(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) <= float(orf[f"orf_score_{orf_caller}"]))):
                            merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] = orf[f"orf_score_{orf_caller}"]
                            merged_orfs_info[orf_key][f"ORF_type_{orf_caller}"] = orf[f"ORF_type_{orf_caller}"]
                            merged_orfs_info[orf_key][f"gene_id_{orf_caller}"] = orf[f"gene_id_{orf_caller}"]
                            merged_orfs_info[orf_key][f"transcript_id_{orf_caller}"] = orf[f"transcript_id_{orf_caller}"]
    return merged_orfs_info

In [ ]:
local_folder= "./data"
orf_callers = ["price", "ribotish", "ribocode"]

merged_orfs_info = {}
count = 0
for experiment_name in open(f"{local_folder}/experiments.txt"):
    experiment_name = experiment_name.split("PRE")[1].strip()[:-1]
    count += load_orfs(experiment_name, local_folder, merged_orfs_info, orf_callers)
print(count) 
pd.DataFrame(merged_orfs_info.values()).reset_index().to_csv(
    f"{local_folder}/merged_orfs_found_by_any_caller.csv",
    sep="\t",
    index=False) 

aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1A/output/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1A/output/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv
aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1B/output/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1B/output/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv


/tmp/ipykernel_1823159/3789791599.py:15: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2A/output/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2A/output/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv


/tmp/ipykernel_1823159/3789791599.py:15: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2B/output/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2B/output/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv


...

aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222241_HEK293T-TG-rep1_SRR8449573/output/VPR_orfcalling_20240307222241_HEK293T-TG-rep1_SRR8449573_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222241_HEK293T-TG-rep1_SRR8449573_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222241_HEK293T-TG-rep1_SRR8449573/output/VPR_orfcalling_20240307222241_HEK293T-TG-rep1_SRR8449573_found_by_any_caller.csv to data/VPR_orfcalling_20240307222241_HEK293T-TG-rep1_SRR8449573_found_by_any_caller.csv


/tmp/ipykernel_1823159/3789791599.py:15: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222241_HEK293T-TG-rep2_SRR8449574/output/VPR_orfcalling_20240307222241_HEK293T-TG-rep2_SRR8449574_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222241_HEK293T-TG-rep2_SRR8449574_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222241_HEK293T-TG-rep2_SRR8449574/output/VPR_orfcalling_20240307222241_HEK293T-TG-rep2_SRR8449574_found_by_any_caller.csv to data/VPR_orfcalling_20240307222241_HEK293T-TG-rep2_SRR8449574_found_by_any_caller.csv


/tmp/ipykernel_1823159/3789791599.py:15: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222241_HEK293T-TM-rep1_SRR8449571/output/VPR_orfcalling_20240307222241_HEK293T-TM-rep1_SRR8449571_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222241_HEK293T-TM-rep1_SRR8449571_found_by_any_caller.csv
